In [10]:
#%autoreload
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
#import imp
#imp.reload(parse_data)


Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [11]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
   # return np.linalg.norm(y_pred-y_true)/np.linalg.norm(y_true)



In [29]:
input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)
added = keras.layers.Add()([x1, x2])  # equivalent to added = keras.layers.add([x1, x2])

out = keras.layers.Dense(4)(added)
model = keras.models.Model(inputs=[input1, input2], outputs=out)

In [32]:
input_rgb = Input((299,299,3))
# RGB VGG model
rgb_model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
#x_rgb = input_x[:,:,:,:3]

#x = Conv2D(3,(3,3), activation='relu', data_format="channels_last")(input_x)
#x = ZeroPadding2D((1,1))(x)
#x = Reshape((299,299,3))(x)
x_rgb = rgb_model_vgg16(input_rgb)

# Depth VGG model
input_depth = Input((299,299,3))
depth_model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
#x_d = input_x[:,:,:,3:]
#print x_d.shape
x_d = depth_model_vgg16(input_depth)

flatten = Flatten()

x = x_rgb + x_d
#x = (np.concatenate((x_rgb,x_d), axis=3))
     
new_layer2 = Dense(1024, activation='relu', name='my_dense_2')
out2 = new_layer2(flatten(x))
x = Dense(512, activation='relu')(out2)
preds = Dense(3, activation = 'relu')(x)

model_start = Model(inputs=[x_rgb, x_d], outputs=preds)
model_start.compile(loss='mean_squared_error', optimizer=SGD(lr=1e-5, momentum=0.9),
            metrics=[root_mean_squared_error])
model_start.summary()


/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:1590: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "model_6" was not an Input tensor, it was generated by layer vgg16.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: vgg16_18/block5_pool/MaxPool:0
  str(x.name))
/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:1590: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "model_6" was not an Input tensor, it was generated by layer vgg16.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: vgg16_19/block5_pool/MaxPool:0
  str(x.name))


AssertionError: 

In [18]:
object_path = "train/48/"
dev_path = "dev/48/"
filename ="vgg"
rescale=1. / 255
img_width, img_height = 299, 299


In [ ]:
dev_num = 7
X_dev = np.empty([0,299,299,4])
y_dev = np.empty([0,3])
for k in range(dev_num):
    x_dev = np.load(dev_path+str(60+k)+"_x.npy") * rescale
    d_dev = np.load(dev_path+str(60+k)+"_d.npy")
    tmp1 = (np.concatenate((x_dev,np.expand_dims(d_dev, axis=3)), axis=3)) 
    X_dev = np.vstack([X_dev, tmp1])
    tmp2 = np.load(dev_path+str(60+k)+"_y.npy")
    y_dev = np.vstack([y_dev, tmp2])
print y_dev.shape
print X_dev.shape

In [ ]:
dev_loss = model_start.evaluate(x = X_dev, y = y_dev)
print "----val_loss & l2_error----:"
print dev_loss


In [ ]:
def train(model):
    filename = "vgg2d_"
    fh = open(filename + 'report.txt','w')
    csv_logger = CSVLogger('logvgg.csv', append=True, separator=';') 
    f_train = open(filename + 'train_report.txt',"a", 0)
    f_train_step = open(filename + 'step_report.txt',"a", 0)
    f_dev = open(filename + 'dev_report.txt',"a", 0)
    batch = 32
    epochs = 20
    batch_num = 60
    rescale=1. / 255
    filepath_best="/home/zhouzixuan/proj/data/weights.best.hdf5"
    if os.path.exists(filepath_best):
        model.load_weights(filepath_best)
        print "load weight success!"
    for e in range(4,epochs):
        print('Epoch', e)
        for b in range(batch_num):
            x_batch = np.load(object_path+str(b)+"_x.npy") * rescale
            d_batch = np.load(object_path+str(b)+"_d.npy")
            X = (np.concatenate((x_batch,np.expand_dims(d_batch, axis=3)), axis=3))           
            y_batch = np.load(object_path+str(b)+"_y.npy")            
            checkpoint = ModelCheckpoint(filepath=filepath_best,monitor='loss', verbose=1,save_best_only=True, mode='min')
            res = model.fit(X, y_batch,callbacks=[checkpoint],verbose=0)
            print str(res.history)
            f_train_step.write(str(res.history))
            f_train_step.write("\n")
        train_loss = model.evaluate(x = X, y = y_batch)
        f_train.write(str(train_loss))
        dev_loss = model.evaluate(x = X_dev, y = y_dev)
        f_dev.write(str(dev_loss))
        print "----val_loss & l2_error----:"
        print dev_loss
    f_train.close()
    f_train_step.close()
    f_dev.close()

In [ ]:
train(model_start)